In [1]:
import polars as pl
import numpy as np
from pathlib import Path

In [3]:
Path().resolve()

PosixPath('/home/kpatelis/projects/bggrecommender')

In [54]:
check=pl.read_csv("bggrecommender/shiny_app/data/bgg_gamelist.csv").filter(pl.col("id") != 170984)
embeddings = np.load("bggrecommender/data/embeddings.npz")["embeddings"]

In [55]:
names = check.select("name").to_series().to_list()
names_arr = check.select("name").to_series().to_numpy()
ids_arr = check.select("id").to_series().to_numpy()

In [31]:
check.filter(pl.col("name").is_in(["Scythe"]))

id,name,rank
i64,str,i64
169786,"""Scythe""",16


In [56]:
embeddings[names_arr == "CATAN"]

array([[ 1.22388499e-02, -8.63511860e-03, -4.93509276e-03,
        -6.76752403e-02, -4.75604320e-03, -5.40627912e-03,
        -3.16062979e-02,  4.44771610e-02, -4.03401740e-02,
        -1.23727974e-02,  7.71993697e-02,  6.34546652e-02,
        -7.11620227e-02, -7.67570287e-02, -2.62582465e-03,
         3.01657189e-02,  3.32320929e-02, -3.27118374e-02,
         2.16350053e-02,  1.65424850e-02,  2.75094081e-02,
         3.79665159e-02, -8.30526184e-03, -1.02965767e-02,
        -1.35255959e-02, -6.03980459e-02,  2.55786460e-02,
         3.34043130e-02, -3.60192321e-02,  3.31228226e-02,
         2.71276776e-02,  6.52049063e-03,  3.88144851e-02,
        -1.37062063e-02,  6.02223054e-02, -1.28230965e-02,
        -1.16595058e-02, -2.12905817e-02,  2.51783617e-02,
        -1.61130838e-02, -2.50524078e-02,  6.13216311e-02,
         4.90262583e-02, -2.93878070e-03,  4.10359837e-02,
         7.98441656e-03,  7.33894035e-02, -5.69206066e-02,
         2.86934394e-02, -1.24059180e-02, -2.80375015e-0

In [40]:
check.select("name").to_series().to_numpy()

array(['Brass: Birmingham', 'Pandemic Legacy: Season 1', 'Gloomhaven',
       ..., 'Candy Land', 'Chutes and Ladders', 'Tic-Tac-Toe'],
      dtype=object)

In [2]:
import shiny
shiny.__version__

'0.4.0'

In [88]:
import requests
import pandas as pd

def retrieve_game_info(game_ids):

    id_list = ','.join(str(game_id) for game_id in game_ids)
    url = f'https://boardgamegeek.com/xmlapi2/thing?id={id_list}&stats=1'
    response = requests.get(url)
    return response.content, response.text

game_ids = [13, 1]  # Replace with your list of game IDs

xml_data, xml_text = retrieve_game_info(game_ids)
#game_data = parse_game_info(xml_data)

#df = pd.DataFrame(game_data)
#print(df)

In [2]:
import requests
from lxml import etree

def retrieve_game_info(game_ids):

    id_list = ','.join(str(game_id) for game_id in game_ids)
    url = f'https://boardgamegeek.com/xmlapi2/thing?id={id_list}&stats=1'
    response = requests.get(url)
    return response.content, response.text # Change when done debugging

def parse_bgg_xml(xml_data):
    
    game_info = []
    
    root = etree.fromstring(xml_data)

    for item in root.xpath('//item'):
        game_name = item.xpath('.//name[@type="primary"]/@value')[0]
        thumbnail = item.xpath('thumbnail/text()')[0]
        image = item.xpath('image/text()')[0]
        description = item.xpath('description/text()')[0]
        year_published = item.xpath('.//yearpublished/@value')[0]
        min_players = item.xpath('.//minplayers/@value')[0]
        max_players = item.xpath('.//maxplayers/@value')[0]
        playing_time = item.xpath('.//playingtime/@value')[0]
        min_playtime = item.xpath('.//minplaytime/@value')[0]
        max_playtime = item.xpath('.//maxplaytime/@value')[0]
        min_age = item.xpath('.//minage/@value')[0]
        average_rating = item.xpath('.//statistics/ratings/average/@value')[0]
        bgg_rating = item.xpath('.//statistics/ratings/bayesaverage/@value')[0]
        
        game_info.append({
            'game_name':game_name, 
            'thumbnail':thumbnail,
            'image':image,
            'description':description,
            'year_published':year_published, 
            'min_players':min_players, 
            'max_players':max_players,
            'playing_time':playing_time,
            'min_playtime':min_playtime,
            'max_playtime':max_playtime,
            'min_age':min_age,
            'average_rating':average_rating, 
            'bgg_rating':bgg_rating
            })
        
    return game_info

In [6]:
game_ids = [63, 11320, 284616, 221, 266769]

xml_data, xml_text = retrieve_game_info(game_ids)

game_data = parse_bgg_xml(xml_data)

game_data[0]["description"]

"(from the back of the box:)&#10;&#10;You are now back in time-12th through 17th century Japan, to be exact. Your objective: the exalted title of &quot;Shogun&quot;. As &quot;Shogun&quot;, you will have the right to enforce the emperor's will, and become the real ruler of Japan.&#10;&#10;Samurai allows you to take control of Japan. In medieval Japan, the central government of the emperor was dissolved. He become a powerless figurehead while various clans competed to take control of the country. &#10;These clans were similar to the noble houses of Europe; and their leaders the equivalent of medieval Dukes, Barons, Counts, and the like. &#10;Using formldable samurai warriors, the clans waged endless wars across the major islands of the Japanese archipelago.&#10;&#10;Concept of Play &#10;Each player controls a clan, with one or more lords. Each lord has a province representing his personal &quot;fief&quot; or home, and often has additional holdings of troops, titles, ships, castles, and m

In [14]:
test=game_data[0]["description"]

In [18]:
test = str(test)

In [25]:
test.encode().decode("ascii", "ignore")

"(from the back of the box:)&#10;&#10;You are now back in time-12th through 17th century Japan, to be exact. Your objective: the exalted title of &quot;Shogun&quot;. As &quot;Shogun&quot;, you will have the right to enforce the emperor's will, and become the real ruler of Japan.&#10;&#10;Samurai allows you to take control of Japan. In medieval Japan, the central government of the emperor was dissolved. He become a powerless figurehead while various clans competed to take control of the country. &#10;These clans were similar to the noble houses of Europe; and their leaders the equivalent of medieval Dukes, Barons, Counts, and the like. &#10;Using formldable samurai warriors, the clans waged endless wars across the major islands of the Japanese archipelago.&#10;&#10;Concept of Play &#10;Each player controls a clan, with one or more lords. Each lord has a province representing his personal &quot;fief&quot; or home, and often has additional holdings of troops, titles, ships, castles, and m

In [36]:
test

"(from the back of the box:)&#10;&#10;You are now back in time-12th through 17th century Japan, to be exact. Your objective: the exalted title of &quot;Shogun&quot;. As &quot;Shogun&quot;, you will have the right to enforce the emperor's will, and become the real ruler of Japan.&#10;&#10;Samurai allows you to take control of Japan. In medieval Japan, the central government of the emperor was dissolved. He become a powerless figurehead while various clans competed to take control of the country. &#10;These clans were similar to the noble houses of Europe; and their leaders the equivalent of medieval Dukes, Barons, Counts, and the like. &#10;Using formldable samurai warriors, the clans waged endless wars across the major islands of the Japanese archipelago.&#10;&#10;Concept of Play &#10;Each player controls a clan, with one or more lords. Each lord has a province representing his personal &quot;fief&quot; or home, and often has additional holdings of troops, titles, ships, castles, and m

In [39]:
import re

test_clean = re.sub(r"(&#10;)+", " ", test)
test_clean = re.sub(r"(&quot;)", '"', test_clean)
test_clean = re.sub(r"(&mdash;)", '-', test_clean)
test_clean = re.sub(r"(&ordm;)", ' degree', test_clean)
test_clean = re.sub(r"(&rdquo;)+", "", test)

test_clean

'(from the back of the box:) You are now back in time-12th through 17th century Japan, to be exact. Your objective: the exalted title of "Shogun". As "Shogun", you will have the right to enforce the emperor\'s will, and become the real ruler of Japan. Samurai allows you to take control of Japan. In medieval Japan, the central government of the emperor was dissolved. He become a powerless figurehead while various clans competed to take control of the country.  These clans were similar to the noble houses of Europe; and their leaders the equivalent of medieval Dukes, Barons, Counts, and the like.  Using formldable samurai warriors, the clans waged endless wars across the major islands of the Japanese archipelago. Concept of Play  Each player controls a clan, with one or more lords. Each lord has a province representing his personal "fief" or home, and often has additional holdings of troops, titles, ships, castles, and more provinces. The game is played in 20 turns, each representing a p

In [40]:
#278, 6778, 27710, 146158, 24083

game_ids = [278, 6778, 27710, 146158, 24083]

xml_data, xml_text = retrieve_game_info(game_ids)

game_data = parse_bgg_xml(xml_data)

name, desc = game_data[0]["game_name"], game_data[0]["description"]

In [45]:
desc_no_name = re.sub(name, "GAME_TOKEN", desc)




desc_no_name

"GAME_TOKEN bears only a slight resemblance to The Settlers of Catan, the original game in the Catan series.  There are six resources: wool, wood, bricks, grain, ore, and gold. Before beginning the game, the players receive six square cards showing the resources, with a different die number on each card. In addition, the players start the game with two village cards and a road card connecting them. There are card stacks of more roads and villages, as well as cities, to be purchased during the game. With each new village or city comes two new resource cards. Villages and cities give victory points.&#10;&#10;When the number die is rolled, instead of collecting more cards, the players rotate the corresponding card 90 degrees to indicate a gain of one more of that resource; i.e., the brick card shows a picture of one brick on one edge of the card, two bricks on the next clockwise edge, three bricks along the third edge, and zero bricks on the fourth edge. There is also an event die, with f

In [47]:
import polars as pl

games=pl.read_csv("bggrecommender/shiny_app/data/bgg_gamelist_all_details.csv").filter(pl.col("id") != 170984)
names_list = games.select("name").to_series().to_list()

In [75]:
names_test = [re.split(" ", name) for name in names_list]

names_test2 = [word for sublist in names_test for word in sublist]

words = pl.Series(name="word", values=names_test2)

In [80]:
words.value_counts(sort=True).filter(pl.col("word")=="Catan")

word,counts
str,u32
"""Catan""",18
